In [1]:
try: import cPickle as pickle
except: import pickle
from os import environ
# if 'DISPLAY' not in environ:
#     import matplotlib
#     matplotlib.use('Agg')

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import itertools
from matplotlib import rc
import numpy as np
import pandas as pd
import seaborn
font = {'family': 'serif', 'serif': ['computer modern roman']}
rc('text', usetex=False)
rc('font', weight='bold')
rc('font', size=8)
rc('lines', markersize=2.5)
rc('lines', linewidth=0.5)
rc('xtick', labelsize=6)
rc('ytick', labelsize=6)
rc('axes', labelsize='small')
rc('axes', labelweight='bold')
rc('axes', titlesize='small')
rc('axes', linewidth=1)
plt.rc('font', **font)
seaborn.set_style("darkgrid")
import pdb

In [2]:
def plot_benchmark(domains, graph_attrs, attr_defaults, methods, graph_names, s_sch='rw'):
    df_all = pd.DataFrame()
    for d, m , g in itertools.product(*[domains, methods, graph_names]):
        try:
            df = pd.read_hdf(path+
                "%s_%s_%s_lp_%s_data_hyp.h5" % (d, g, m, s_sch),
                "df"
            )
        except:
            print('%s_%s_%s_lp_%s_data_hyp.h5 not found. Ignoring data set' % (d, g, m, s_sch))
            continue
        df["Domain"], df["Method"], df["Graph"] = d, m, g
        df_all = df_all.append(df).reset_index()
        df_all = df_all.drop(['index'], axis=1)
    if df_all.empty:
        return
    df_all = df_all.drop(['dia'], axis=1)
    plot_shape = (len(domains), len(graph_attrs))
    fin1, axarray1 = plt.subplots(len(domains), len(graph_attrs), figsize=(7, 4), sharex='col', sharey='row')
    data_idx = 0
    gfs_score = {}
    for dom in domains:
        gfs_score[dom] = {m: 0 for m in methods}
        n_attr = 0
        for attr in graph_attrs:
            plot_idx = np.unravel_index(data_idx, plot_shape)
            data_idx += 1
#             df_trun = df_all[graph_attrs + [hyp_key_ren, "Round Id", expMap[exp], "Data"]]
            try:
#                 pdb.set_trace()
                rem_attrs = list(set(graph_attrs) - {attr})
                df_grouped = df_all[df_all["Domain"]==dom]
                for rem_attr in rem_attrs:
                    df_grouped = df_grouped[df_grouped[rem_attr]==attr_defaults[rem_attr]]
                df_grouped = df_grouped[[attr, "Round Id", "LP MAP", "Method", "Graph"]]
                print(df_grouped.head())
                df_grouped['LP MAP'] = df_grouped['LP MAP'].astype('float')
                df_grouped = df_grouped.groupby([attr, "Round Id", "Method", "Graph"]).mean().reset_index()
            except TypeError:
                df_trun[hyp_key_ren + "2"] = \
                    df_trun[hyp_key_ren].apply(lambda x: str(x))
                df_trun[hyp_key_ren] = df_trun[hyp_key_ren + "2"].copy()
                df_trun = df_trun.drop([hyp_key_ren + "2"], axis=1)
                df_grouped = df_trun.groupby([hyp_key_ren, "Round Id", "Data"]).max().reset_index()  

#             df_grouped = df_grouped.drop(['Graph'], axis=1).reset_index().drop(['index'], axis=1)
#             pdb.set_trace()
            #print(df_grouped)
            df_grouped['unit']=df_grouped.apply(lambda x:'%s_%s' % (x['Round Id'],x['Graph']),axis=1)
            df_grouped = df_grouped.drop(['Round Id', "Graph"], axis=1)
            m_scores = dict(df_grouped.groupby(["Method"])["LP MAP"].mean())
            n_attr += 1
            gfs_score[dom] = {m: m_scores[m] + gfs_score[dom][m] for m in methods}
            ax = seaborn.tsplot(time=attr, value="LP MAP",
                                    unit="unit", condition="Method", legend=False,
                                    data=df_grouped, ax=axarray1[plot_idx[0], plot_idx[1]])
            if not plot_idx[1]:
                ax.set_ylabel(dom)
            if plot_idx[1]:
                ax.set_ylabel('')
            if plot_idx[0] < len(domains) - 1:
                ax.set_xlabel('')
            attr_values = df_grouped[attr].unique()
            l_diff = attr_values[-1] - attr_values[-2]
            f_diff = attr_values[1] - attr_values[0]
            l_f_diff_r = l_diff / f_diff
            if l_f_diff_r > 1:
                log_base = pow(l_f_diff_r, 1.0 / (len(attr_values) - 2))
                ax.set_xscale('log', basex=round(log_base))
            marker = ["o", "s", "D", "^", "v", "8", "*", "p", "1", "h"]
            for line_i in range(len(ax.lines)):
                ax.lines[line_i].set_marker(marker[line_i])
            # ax.grid()
#             ax.legend()
        gfs_score[dom] = {m: gfs_score[dom][m]/n_attr for m in methods}
    print(gfs_score)
    plt.savefig(
       'benchmark.pdf', # gem/plots/hyp/
       dpi=300, format='pdf', bbox_inches='tight'
    )
    plt.show()
#     plt.clf()

In [3]:
def plot_benchmark_individual(domains, graph_attrs, attr_defaults, methods, graph_names, graph_short, s_sch='rw'):
    df_all = pd.DataFrame()
    for d, m , g in itertools.product(*[domains, methods, graph_names]):
        try:
            df = pd.read_hdf(path+
                "%s_%s_%s_lp_%s_data_hyp.h5" % (d, g, m, s_sch),
                "df"
            )
        except:
            print('%s_%s_%s_lp_%s_data_hyp.h5 not found. Ignoring data set' % (d, g, m, s_sch))
            continue
        df["Method"], df["Graph"] = m, g
        df_all = df_all.append(df).reset_index()
        df_all = df_all.drop(['index'], axis=1)
    if df_all.empty:
        return
    df_all = df_all.drop(['dia'], axis=1)
    plot_shape = (len(graph_names), len(graph_attrs))
    fin1, axarray1 = plt.subplots(len(graph_names), len(graph_attrs), figsize=(7, 4), sharex='col', sharey='row')
    data_idx = 0
    for graph in graph_names:
        for attr in graph_attrs:
            plot_idx = np.unravel_index(data_idx, plot_shape)
            data_idx += 1
#             df_trun = df_all[graph_attrs + [hyp_key_ren, "Round Id", expMap[exp], "Data"]]
            try:
#                 pdb.set_trace()
                rem_attrs = list(set(graph_attrs) - {attr})
                df_grouped = df_all[df_all["Graph"]==graph]
                for rem_attr in rem_attrs:
                    df_grouped = df_grouped[df_grouped[rem_attr]==attr_defaults[rem_attr]]
                df_grouped = df_grouped[[attr, "Round Id", "LP MAP", "Method"]]
                print(df_grouped.head())
                df_grouped['LP MAP'] = df_grouped['LP MAP'].astype('float')
                df_grouped = df_grouped.groupby([attr, "Round Id", "Method"]).mean().reset_index()
            except TypeError:
                df_trun[hyp_key_ren + "2"] = \
                    df_trun[hyp_key_ren].apply(lambda x: str(x))
                df_trun[hyp_key_ren] = df_trun[hyp_key_ren + "2"].copy()
                df_trun = df_trun.drop([hyp_key_ren + "2"], axis=1)
                df_grouped = df_trun.groupby([hyp_key_ren, "Round Id", "Data"]).max().reset_index()  

#             df_grouped = df_grouped.drop(['Graph'], axis=1).reset_index().drop(['index'], axis=1)
#             pdb.set_trace()
            #print(df_grouped)
            if data_idx == len(graph_names) * len(graph_attrs):
                legend = True
            else:
                legend = False
            ax = seaborn.tsplot(time=attr, value="LP MAP",
                                    unit="Round Id", condition="Method", legend=legend,
                                    data=df_grouped, ax=axarray1[plot_idx[0], plot_idx[1]])
            if legend:
                ax.legend_.remove()
            if not plot_idx[1]:
                ax.set_ylabel(graph_short[graph])
            if plot_idx[1]:
                ax.set_ylabel('')
            if plot_idx[0] < len(graph_names) - 1:
                ax.set_xlabel('')
            attr_values = df_grouped[attr].unique()
            l_diff = attr_values[-1] - attr_values[-2]
            f_diff = attr_values[1] - attr_values[0]
            l_f_diff_r = l_diff / f_diff
            if l_f_diff_r > 1:
                log_base = pow(l_f_diff_r, 1.0 / (len(attr_values) - 2))
                ax.set_xscale('log', basex=round(log_base))
            marker = ["o", "s", "D", "^", "v", "8", "*", "p", "1", "h"]
            for line_i in range(len(ax.lines)):
                ax.lines[line_i].set_marker(marker[line_i])
            # ax.grid()
#             ax.legend()
    for col_idx in range(axarray1[3].shape[0]):
        box = axarray1[3][col_idx].get_position()
        axarray1[3][col_idx].set_position(
            [box.x0,
             box.y0 + box.height * 0.1,
             box.width,
             box.height * 0.9]
        )
    fin1.legend(loc='lower center', bbox_to_anchor=(0.45, -0.01),
                ncol=len(methods), fancybox=True, shadow=True)
    fin1.savefig(
       'benchmark_individual3.pdf', # gem/plots/hyp/
       dpi=300, format='pdf', bbox_inches='tight'
    )
    plt.show()
#     plt.clf()

In [4]:
path = '/Users/HuangDi0422/Documents/GitHub/HPC_syn/Bench_files/new_files/'
#path = '/Users/HuangDi0422/Documents/GitHub/HPC_syn/Bench_files/'

In [5]:
plot_benchmark(
    ["social", "biology", "internet"],
    ["N", "dim", "deg"],
    {"N": 4096, "deg": 8, "dia": None, "dim": 128},
    ["gf", "rand", "pa", "lap", "hope", "cn", "aa","sdne"],
    ["watts_strogatz_graph", "barabasi_albert_graph", "powerlaw_cluster_graph", "random_geometric_graph",\
     "duplication_divergence_graph","hyperbolic_graph","r_mat_graph","waxman_graph","stochastic_block_model",\
     "stochastic_kronecker_graph"],
    s_sch='rw'
)

social_watts_strogatz_graph_gf_lp_rw_data_hyp.h5 not found. Ignoring data set
social_barabasi_albert_graph_gf_lp_rw_data_hyp.h5 not found. Ignoring data set
social_powerlaw_cluster_graph_gf_lp_rw_data_hyp.h5 not found. Ignoring data set
social_duplication_divergence_graph_gf_lp_rw_data_hyp.h5 not found. Ignoring data set
social_hyperbolic_graph_gf_lp_rw_data_hyp.h5 not found. Ignoring data set
social_r_mat_graph_gf_lp_rw_data_hyp.h5 not found. Ignoring data set
social_watts_strogatz_graph_rand_lp_rw_data_hyp.h5 not found. Ignoring data set
social_barabasi_albert_graph_rand_lp_rw_data_hyp.h5 not found. Ignoring data set
social_powerlaw_cluster_graph_rand_lp_rw_data_hyp.h5 not found. Ignoring data set
social_duplication_divergence_graph_rand_lp_rw_data_hyp.h5 not found. Ignoring data set
social_hyperbolic_graph_rand_lp_rw_data_hyp.h5 not found. Ignoring data set
social_r_mat_graph_rand_lp_rw_data_hyp.h5 not found. Ignoring data set
social_watts_strogatz_graph_pa_lp_rw_data_hyp.h5 not fou

internet_waxman_graph_hope_lp_rw_data_hyp.h5 not found. Ignoring data set
internet_stochastic_block_model_hope_lp_rw_data_hyp.h5 not found. Ignoring data set
internet_watts_strogatz_graph_cn_lp_rw_data_hyp.h5 not found. Ignoring data set
internet_random_geometric_graph_cn_lp_rw_data_hyp.h5 not found. Ignoring data set
internet_duplication_divergence_graph_cn_lp_rw_data_hyp.h5 not found. Ignoring data set
internet_hyperbolic_graph_cn_lp_rw_data_hyp.h5 not found. Ignoring data set
internet_waxman_graph_cn_lp_rw_data_hyp.h5 not found. Ignoring data set
internet_stochastic_block_model_cn_lp_rw_data_hyp.h5 not found. Ignoring data set
internet_watts_strogatz_graph_aa_lp_rw_data_hyp.h5 not found. Ignoring data set
internet_random_geometric_graph_aa_lp_rw_data_hyp.h5 not found. Ignoring data set
internet_duplication_divergence_graph_aa_lp_rw_data_hyp.h5 not found. Ignoring data set
internet_hyperbolic_graph_aa_lp_rw_data_hyp.h5 not found. Ignoring data set
internet_waxman_graph_aa_lp_rw_data_

/Users/HuangDi0422/anaconda/lib/python3.5/site-packages/seaborn-0.9.0-py3.5.egg/seaborn/timeseries.py:183: UserWarning: The `tsplot` function is deprecated and will be removed in a future release. Please update your code to use the new `lineplot` function.
  warnings.warn(msg, UserWarning)
/Users/HuangDi0422/anaconda/lib/python3.5/site-packages/scipy/stats/stats.py:1713: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.add.reduce(sorted[indexer] * weights, axis=axis) / sumval


    dim Round Id    LP MAP Method                   Graph
10  128        0  0.295225     gf  random_geometric_graph
11  128        1  0.189768     gf  random_geometric_graph
12  128        2  0.192935     gf  random_geometric_graph
13  128        3  0.199691     gf  random_geometric_graph
14  128        4  0.189906     gf  random_geometric_graph
   deg Round Id    LP MAP Method                   Graph
10   8        0  0.295225     gf  random_geometric_graph
11   8        1  0.189768     gf  random_geometric_graph
12   8        2  0.192935     gf  random_geometric_graph
13   8        3  0.199691     gf  random_geometric_graph
14   8        4  0.189906     gf  random_geometric_graph
         N Round Id    LP MAP Method                 Graph
2080  1024        0  0.350719     gf  watts_strogatz_graph
2081  1024        1  0.345384     gf  watts_strogatz_graph
2082  1024        2  0.336399     gf  watts_strogatz_graph
2083  1024        3  0.353616     gf  watts_strogatz_graph
2084  1024     

/Users/HuangDi0422/anaconda/lib/python3.5/site-packages/matplotlib/figure.py:445: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  % get_backend())


In [7]:
plot_benchmark_individual(
    ["social", "biology", "internet"],
    ["N", "dim", "deg"],
    {"N": 4096, "deg": 8, "dia": None, "dim": 128},
    ["gf", "rand", "pa", "lap", "hope", "cn", "aa", "sdne"],
    ["watts_strogatz_graph", "barabasi_albert_graph", "powerlaw_cluster_graph", "random_geometric_graph",\
     "duplication_divergence_graph","hyperbolic_graph","r_mat_graph","waxman_graph","stochastic_kronecker_graph"],
    {"watts_strogatz_graph": "WS", "barabasi_albert_graph": "BA", \
     "powerlaw_cluster_graph": "PC", "random_geometric_graph": "RG",\
     "duplication_divergence_graph": "DD", "hyperbolic_graph": "HB",\
     "r_mat_graph": "RM", "waxman_graph":"WM","stochastic_block_model":"SBM", "stochastic_kronecker_graph": "KG"
    },
    s_sch='rw'
)

social_watts_strogatz_graph_gf_lp_rw_data_hyp.h5 not found. Ignoring data set
social_barabasi_albert_graph_gf_lp_rw_data_hyp.h5 not found. Ignoring data set
social_powerlaw_cluster_graph_gf_lp_rw_data_hyp.h5 not found. Ignoring data set
social_duplication_divergence_graph_gf_lp_rw_data_hyp.h5 not found. Ignoring data set
social_hyperbolic_graph_gf_lp_rw_data_hyp.h5 not found. Ignoring data set
social_r_mat_graph_gf_lp_rw_data_hyp.h5 not found. Ignoring data set
social_watts_strogatz_graph_rand_lp_rw_data_hyp.h5 not found. Ignoring data set
social_barabasi_albert_graph_rand_lp_rw_data_hyp.h5 not found. Ignoring data set
social_powerlaw_cluster_graph_rand_lp_rw_data_hyp.h5 not found. Ignoring data set
social_duplication_divergence_graph_rand_lp_rw_data_hyp.h5 not found. Ignoring data set
social_hyperbolic_graph_rand_lp_rw_data_hyp.h5 not found. Ignoring data set
social_r_mat_graph_rand_lp_rw_data_hyp.h5 not found. Ignoring data set
social_watts_strogatz_graph_pa_lp_rw_data_hyp.h5 not fou

internet_random_geometric_graph_lap_lp_rw_data_hyp.h5 not found. Ignoring data set
internet_duplication_divergence_graph_lap_lp_rw_data_hyp.h5 not found. Ignoring data set
internet_hyperbolic_graph_lap_lp_rw_data_hyp.h5 not found. Ignoring data set
internet_waxman_graph_lap_lp_rw_data_hyp.h5 not found. Ignoring data set
internet_watts_strogatz_graph_hope_lp_rw_data_hyp.h5 not found. Ignoring data set
internet_random_geometric_graph_hope_lp_rw_data_hyp.h5 not found. Ignoring data set
internet_duplication_divergence_graph_hope_lp_rw_data_hyp.h5 not found. Ignoring data set
internet_hyperbolic_graph_hope_lp_rw_data_hyp.h5 not found. Ignoring data set
internet_waxman_graph_hope_lp_rw_data_hyp.h5 not found. Ignoring data set
internet_watts_strogatz_graph_cn_lp_rw_data_hyp.h5 not found. Ignoring data set
internet_random_geometric_graph_cn_lp_rw_data_hyp.h5 not found. Ignoring data set
internet_duplication_divergence_graph_cn_lp_rw_data_hyp.h5 not found. Ignoring data set
internet_hyperbolic_

/Users/HuangDi0422/anaconda/lib/python3.5/site-packages/seaborn-0.9.0-py3.5.egg/seaborn/timeseries.py:183: UserWarning: The `tsplot` function is deprecated and will be removed in a future release. Please update your code to use the new `lineplot` function.
  warnings.warn(msg, UserWarning)
/Users/HuangDi0422/anaconda/lib/python3.5/site-packages/scipy/stats/stats.py:1713: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.add.reduce(sorted[indexer] * weights, axis=axis) / sumval


      dim Round Id    LP MAP Method
1570  128        0  0.332041     gf
1571  128        1  0.348013     gf
1572  128        2   0.35154     gf
1573  128        3  0.332422     gf
1574  128        4  0.347637     gf
     deg Round Id    LP MAP Method
1570   8        0  0.332041     gf
1571   8        1  0.348013     gf
1572   8        2   0.35154     gf
1573   8        3  0.332422     gf
1574   8        4  0.347637     gf
         N Round Id      LP MAP Method
3575  1024        0  0.00842129     gf
3576  1024        1  0.00889223     gf
3577  1024        2  0.00667578     gf
3578  1024        3   0.0088913     gf
3579  1024        4  0.00897508     gf
      dim Round Id      LP MAP Method
3585  128        0  0.00194378     gf
3586  128        1   0.0018569     gf
3587  128        2  0.00198637     gf
3588  128        3  0.00248175     gf
3589  128        4  0.00324453     gf
     deg Round Id      LP MAP Method
3585   8        0  0.00194378     gf
3586   8        1   0.0018569     gf
3

/Users/HuangDi0422/anaconda/lib/python3.5/site-packages/matplotlib/figure.py:445: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  % get_backend())
